!git clone https://www.modelscope.cn/datasets/BJQW14B/bs_challenge_financial_14b_dataset.git

In [22]:
import sqlite3

# 创建连接
conn = sqlite3.connect('/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/db/bojin.db')

cursor = conn.cursor()

# # 创建表
# cursor.execute('''CREATE TABLE yourTableName
#                   (column1 datatype, column2 datatype)''')

# # 插入数据
# cursor.execute("INSERT INTO yourTableName VALUES (value1, value2)")

# # 提交改动
# conn.commit()

# 查询数据
cursor.execute("""
SELECT AVG(B."成交金额(元)") AS "平均成交金额"
FROM "A股公司行业划分表" AS A
JOIN "A股票日行情表" AS B ON A."股票代码" = B."股票代码" and A."交易日期" = B."交易日"
WHERE A."行业划分标准" = '申万行业分类'
AND A."二级行业名称" = '银行' 
AND B."交易日" = '20190118';
""")
results = cursor.fetchall() 
for row in results:
    print(row)
# (21012,)

# cursor.execute("""
# SELECT *
#     FROM "A股公司行业划分表"
#     WHERE "行业划分标准" = '中信行业分类'
#     AND "二级行业名称" = '其他医药医疗'
#     limit 10
# """)
# results = cursor.fetchall()
# for row in results:
#     print(row)
#     1253
cursor.close()
conn.close()

(456321251.1,)


In [ ]:
import jsonlines

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content


def write_jsonl(path, content):
    with jsonlines.open(path, "w") as json_file:
        json_file.write_all(content)


content = read_jsonl("/Users/jxy/Projects/challenge/bs_challenge_financial_14b_dataset/question.json")


In [ ]:
for cont in content:
    cont['answer'] = "unknown"
write_jsonl("/Users/jxy/Projects/challenge/bs_challenge_financial_14b_dataset/submit_result.jsonl", content=content)

# try pypdf

In [ ]:
import re
# 页眉页脚
pattern = r'[\s\n]*(.*招\s?股\s?意向书.*)?[\s\n]+[\d-]*[\s\n]*'

from langchain.document_loaders import *

import os
pdfpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/"
txtpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/newtxt/"
filepaths = os.listdir(pdfpath)
filepaths.sort()
print("total file="  + str(len(filepaths)))

for idx, filename in enumerate(filepaths):
    print("begin i=" + str(idx) + " filename= " + filename)
    flagSub = False
    # first PyPDFLoader ， then PyMuPDFLoader
    with open(txtpath + filename.replace("PDF", "txt"), "w") as out:
        try:
            if filename == "07d29cd67ca8e0fc932e05178db1fcdca1cee937.PDF": # 繁体字
                docs = PyMuPDFLoader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/" + filename).load()
            else:
                docs = PyPDFLoader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/" + filename).load()
        except Exception as e:
            print("error i=" + str(idx) + " filename= " + filename)
            docs = PyMuPDFLoader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/" + filename).load()
        for doc in docs:
            match = re.match(pattern, doc.page_content)
            if match and match.end() < 100:
                out.write(doc.page_content[match.end():]) # ignore page header
            else:
                out.write(doc.page_content)
            # only for head
            # max_len = len(doc.page_content)
            # head = ""
            # rest = ""
            # if doc.metadata['page'] != 0: # 忽略第一页
            #     if max_len > 150:
            #         head = doc.page_content[0:150]
            #         rest = doc.page_content[150:]
            #     else:
            #         head = doc.page_content
            #     res_head = re.sub(pattern, "", head)
            #     if len(res_head) != len(head):
            #         flagSub = True
            #     out.write(res_head)
            #     out.write(rest)
            # else:
            #     out.write(doc.page_content)
        print("end i =" + str(idx) + ", filename= " +filename + " flagSub=" + str(flagSub))
print("end")

In [ ]:
from PyPDF2 import PdfReader
reader = PdfReader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF")
page = reader.pages[9]
print(page.extract_text((0,90)))

In [ ]:
# 蛮快的，并且质量还行，没有异常停止
import pdfplumber
docs = []
pdffiles = [
    "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/50b2823371fe1699d260f67cadac3d38af0672e3.PDF"
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/f587290218d881e18e88fc1431b022b2c5aca81a.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/f9e84ce0edd5279773b3ca1f36a9e39d6ceaf220.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/d336d607e1d431cbfe1f313e2234a13fcf49a16e.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/a6f8156c08a1096c46470a1c5e1229daaaedf06e.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/54d148902b889679830174597830f0d0f22c1073.PDF",
]
for file_path in pdffiles:
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
        # Extract the text
        # first_page = pdf.pages[0]
            # print(page.extract_text())
            # print("\n\n")
        # Extract the data
            # print(page.extract_text_lines()) 
            tables = page.extract_table()
            if tables is not None:
                print("\ntext:\n")
                print(page.extract_text(layout=True))  
                print("\ntables:\n")
                for table in tables:
                    print(str(table).replace("\\n", ""))
                break;
            images = page.extract_images()
            if images is not None:
                print("\nimages:\n")
                for image in images:
                    print(image)
                break;
        # # Extract the images
        # images = pdf.get_images()
        # for image in images:
        #     print(image["page_number"])
            # with open(f"image_{image['page_number']}.jpg", "wb") as f:
            #     f.write(image["data"])


In [ ]:
# PyMuPDF 质量一般
import sys, pathlib, fitz
for file_path in pdffiles:
    with fitz.open(file_path) as doc:  # open document
        # text = chr(12).join([page.get_text() for page in doc])
        page = doc[9]
        tables = page.find_tables()
        print(doc[9].get_text())
        print("\ntables:\n")
        for table in tables:
            print(str(table).replace("\\n", ""))
        # print(text[0:3000])
    # write as a binary file to support non-ASCII characters
    # pathlib.Path(fname + ".txt").write_bytes(text.encode())

In [ ]:
# bord质量还行，但是会抛异常
import typing
import sys, pathlib
from borb.pdf import Document
from borb.pdf import PDF
from borb.toolkit import SimpleTextExtraction
outpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/newtxt/a.txt"
# for file_path in pdffiles:
file_path = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/50b2823371fe1699d260f67cadac3d38af0672e3.PDF"
# read the Document
doc: typing.Optional[Document] = None
l: SimpleTextExtraction = SimpleTextExtraction()
with open(file_path, "rb") as in_file_handle:
    doc = PDF.loads(in_file_handle, [l])

    # check whether we have read a Document
    # assert doc is not None

    # print the text on the first Page
# print(doc)
# print(l.get_text(0))
print(l.get_text()(0))
# for page in l.get_text().pages:
    # print(page)
    # break;
    # pathlib.Path(outpath).write_bytes(page)

In [ ]:
l.get_text()[1]

In [ ]:
# 表格处理
from langchain.document_loaders import *
# PyPDFLoader PyPDFium2Loader PDFMinerLoader PyMuPDFLoader PDFPlumberLoader MathpixPDFLoader 
filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF"
loaders = [
    PyPDFLoader(filepath), # 比较好 \n
    # PyPDFium2Loader(filepath), # 一般, \r\r
    # PDFMinerLoader(filepath), # 只有一页
    PyMuPDFLoader(filepath), # 一般 \n
    # PDFPlumberLoader(filepath),  # table 处理比较乱
    # MathpixPDFLoader(filepath), # 需要其他环境依赖
]
for loader in loaders:
    docs = loader.load();
    print(docs[9])


In [ ]:
# 数据异常
import re
pattern = r'[\s\n]*(.*招股\s?意向书.*)?[\s\n]*[\d-]+[\s\n]*'




from langchain.document_loaders import *
# PyPDFLoader PyPDFium2Loader PDFMinerLoader PyMuPDFLoader PDFPlumberLoader MathpixPDFLoader 
# filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/9951262c20ed33562a2fff85c83aeae320f14922.PDF"
filepath= "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/03c625c108ac0137f413dfd4136adb55c74b3805.PDF"
# filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/334a4deb4169c220c7a89243a104f6874ce1996a.PDF"
loaders = [
    PyPDFLoader(filepath), # 比较好 \n
    # PyPDFium2Loader(filepath), # 一般, \r\r
    # PDFMinerLoader(filepath), # 只有一页
    # PyMuPDFLoader(filepath), # 一般 \n
    # PDFPlumberLoader(filepath),  # table 处理比较乱
    # MathpixPDFLoader(filepath), # 需要其他环境依赖
]
for loader in loaders:
    docs = loader.load();
    print(docs[0])
    match1 = re.match(pattern, docs[0].page_content)
    if match1:
        print("match1 = " + str(match1))
        print("result1=" + docs[0].page_content[match1.end():])
    
    print(docs[1])
    match2 = re.match(pattern, docs[1].page_content)
    if match2:
        print("match2 = " + str(match2))
        print("result2=" + docs[1].page_content[match2.end():])   

In [ ]:
# 乱码处理：是繁体字导致
# 无法读出第一页

# /Users/jxy/Projects/person/ChallengeAI/2023bojin/data/newtxt/afa8c5a4a91c3ecf7bd38a1c1f09b8a68e472909.txt


from langchain.document_loaders import *
# PyPDFLoader PyPDFium2Loader PDFMinerLoader PyMuPDFLoader PDFPlumberLoader MathpixPDFLoader 
filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/50b2823371fe1699d260f67cadac3d38af0672e3.PDF"
# 第一页读不出来 filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/a6f8156c08a1096c46470a1c5e1229daaaedf06e.PDF"
# 第一页读不出来 filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/d6c70c04287ff6aba75b0565bef375469aaa68c3.PDF"
# 前面有部分无法读出来 ，显示一个 A filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/afa8c5a4a91c3ecf7bd38a1c1f09b8a68e472909.PDF"
# filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/afa8c5a4a91c3ecf7bd38a1c1f09b8a68e472909.PDF" 繁体
loaders = [
    PyPDFLoader(filepath), # 比较好 \n
    # PyPDFium2Loader(filepath), # 一般, \r\r
    # PDFMinerLoader(filepath), # 只有一页
    # PyMuPDFLoader(filepath), # 一般 \n
    # PDFPlumberLoader(filepath),  # table 处理比较乱
    # MathpixPDFLoader(filepath), # 需要其他环境依赖
]
print("begin")
for loader in loaders:
    # print(loader.getDocumentInfo().title.encode("big5"))
    docs = loader.load();
    # print(docs[0].page_content.encode('big5'))
    print(docs[0].page_content)
    print(docs[1].page_content)
    # print(docs[2].page_content)